In [34]:
# Example use of the Sonic module Sputnik.jl

See https://julialang.org, https://juliadb.org and https://juliastats.org for reference.

In [2]:
import Pkg                      # package manager
Pkg.activate("..")              # Project.toml
Pkg.instantiate()               # https://julialang.github.io/Pkg.jl/v1.0/index.html

Activating environment at `~/Documents/Calendar/Work/Noda/Code/Sonic.jl/Project.toml`


In [32]:
import Dates, JuliaDB, Plots, Random, StatsPlots

In [4]:
using StatsPlots
ENV["GKSwstype"] = "100"
gr(fmt = :png)

Plots.GRBackend()

Get secret token,

In [5]:
include("../src/LinckiiSecrets.jl")
secret = LinckiiSecrets.keab

(url = "https://canary.noda.se/~keab", key = "e86d93131e6bc12ae701d036e696cf8b")

In [6]:
include("../src/Linckii.jl")
include("../src/Sputnik.jl")

Main.Sputnik

Load site,

In [7]:
@time site = Linckii.loadsite(secret, :nodes, :devices, :sensors)
map(length, site)

 28.736444 seconds (25.10 M allocations: 1.683 GiB, 1.62% gc time)


(nodes = 279, devices = 11, sensors = 175)

Specify patterns,

In [8]:
patterns = Dict(
    4 => Dict(
        :meter_effect           => identity,
        :meter_heatenergy       => identity,
        :meter_primreturntemp   => identity,
        :meter_primsupplytemp   => identity,
        :meter_volume           => identity,
        :meter_volumeflow       => identity,
        :outdoortemp            => identity,
        :outdoortemp_offset     => identity,
    ),
    6 => Dict(
        :cloudiness             => identity,
        :outdoortemp            => identity,
        :wind_direction         => identity,
        :wind_speed             => identity,
    ),
)

Dict{Int64,Dict{Symbol,V} where V} with 2 entries:
  4 => Dict{Symbol,Function}(:meter_primreturntemp=>#9,:meter_effect=>#9,:outdo…
  6 => Dict(:cloudiness=>#9,:outdoortemp=>#9,:wind_direction=>#9,:wind_speed=>#…

Get pattern nodes,

In [13]:
rs = JuliaDB.rows(Linckii.get_pattern_nodes(patterns; site...))
if true
    rs = [(node_id = 62, device_id = 4), (node_id = 93, device_id = 4)] # [rs[Random.rand(1 : end)]]
end
length(rs)

2-element Array{NamedTuple{(:node_id, :device_id),Tuple{Int64,Int64}},1}:
 (node_id = 62, device_id = 4)
 (node_id = 93, device_id = 4)

Load data,

In [14]:
data = Dict()

Dict{Any,Any} with 0 entries

In [45]:
# https://discourse.julialang.org/t/reading-data-is-still-too-slow/17919
@time for r in rs
    r = Linckii.dbroot(secret)
    p = Linckii.dbpath(r, "data"; db = r.node_id)
    try
        data[r.node_id] = JuliaDB.dropmissing(JuliaDB.load(p))
        println("$(p): Done")
    catch e
        if typeof(e) <: InterruptException
            rethrow(e)
        end
        println("$(p): $(e)")
    end
end

db/linckii/~keab/data/62.db: Done
db/linckii/~keab/data/93.db: Done
  0.353706 seconds (2.76 M allocations: 184.608 MiB, 14.77% gc time)


## HEKA
* Primary differential temperature 
* Volume weighted primary differential temperature
* Primary return temperature
* Volume weighted primary return temperature
* Volume per energy
* Overflow
* Balance temperature 
* Domestic hot water quota
* Energy signature

### Primary differential temperature

In [ ]:
d = Dict()

In [59]:
@time for r in rs
    cs = JuliaDB.columns(data[r.node_id])
    d[r.node_id] = mean(cs.meter_primsupplytemp .- cs.meter_primreturntemp)
end

UndefVarError: UndefVarError: mean not defined

In [60]:
d

Dict{Any,Any} with 2 entries:
  93 => 54.6785
  62 => 57.0943

### Volume weighted primary differential temperature

In [54]:
d = Dict()

Dict{Any,Any} with 0 entries

In [57]:
@time for r in rs
    cs = JuliaDB.columns(data[r.node_id])
    d[r.node_id] = sum(
        (cs.meter_primsupplytemp .- cs.meter_primreturntemp) .* cs.meter_volumeflow
    ) / sum(cs.meter_volumeflow)
end

  0.000478 seconds (28 allocations: 666.469 KiB)


In [58]:
d

Dict{Any,Any} with 2 entries:
  93 => 54.6785
  62 => 57.0943

### Primary return temperature

### Volume weighted primary return temperature

### Volume per energy

### Overflow

### Balance temperature 

### Domestic hot water quota

### Energy signature